In [41]:
import os
os.environ["TOGETHER_API_KEY"] = "148521c4088ad416dced465cc144671626b00c860af4e6ebc855953567087d8a"
os.environ["COHERE_API_KEY"] = 'xxe3X6u8vcTFJgJ8Pc7CfLezwpQiATQcUB56VIUp'

TOP_K = 5
MAX_DOCS_FOR_CONTEXT = 5

In [42]:
from operator import itemgetter
from langchain.vectorstores import Qdrant
from langchain.llms import Cohere
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from qdrant_client.http import models
from langchain_core.documents.base import Document
from langchain.load import dumps, loads
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.retrievers import BaseRetriever
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from sentence_transformers.util import cos_sim
import PyPDF2
from langchain.embeddings.cohere import CohereEmbeddings
from langchain_community.document_loaders import PyPDFLoader

In [43]:
qdrant_client = QdrantClient(
    "https://8803fa99-7551-4f88-84c3-e134c9bed5de.us-east4-0.gcp.cloud.qdrant.io:6333",
    prefer_grpc=True,
    api_key="EFeN_UhdmAlDNYZHqJBUbZ88Nt7N0MkmvWLgM5Hs4ogNvExLMwNwdQ",
)

QDRANT_URL = "https://8803fa99-7551-4f88-84c3-e134c9bed5de.us-east4-0.gcp.cloud.qdrant.io:6333"
QDRANT_API_KEY = "EFeN_UhdmAlDNYZHqJBUbZ88Nt7N0MkmvWLgM5Hs4ogNvExLMwNwdQ"

In [44]:
def get_prompt(counter, total_docs):
    threshold = total_docs // 2
    template = None

    if counter > threshold:
        template = """
                Create several question based on question:{query}, context: {context} and chat history of the user: {chat_history}.
                As similarity between query and context is low, try to ask several probing questions.
                Ask several followup questions to get further clarity.
                Answer in a polite tone, and convey to the user that you need more clarity to answer the question.
                Then display the probing questions as bulletin points.
                Do not use technical words, give easy to understand responses.
            """
    else:
        template = """
                Answer the question:{query} only based on the context: {context} and the chat history of the user: {chat_history} provided.
                Try to answer in bulletin points.
                Do not use technical words, give easy to understand responses.
                Do not divulge any other details other than query or context.
            """
        
    return template

In [45]:
def reciprocal_rank_fusion(results: list[list], k=60):
    """Rerank docs (Reciprocal Rank Fusion)

    Args:
        results (list[list]): retrieved documents
        k (int, optional): parameter k for RRF. Defaults to 60.

    Returns:
        ranked_results: list of documents reranked by RRF
    """

    print("results in rrf function: ", results)

    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # return only documents
    return [x[0] for x in reranked_results[:MAX_DOCS_FOR_CONTEXT]]

In [46]:
def query_generator(original_query: dict) -> list[str]:
    """Generate queries from original query

    Args:
        query (dict): original query

    Returns:
        list[str]: list of generated queries
    """

    # original query
    query = original_query.get("query")

    # prompt for query generator
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that generates multiple search queries based on a single input query."),
        ("user", "Generate multiple search queries related to:  {original_query}. When creating queries, please refine or add closely related contextual information, without significantly altering the original query's meaning"),
        ("user", "OUTPUT (3 queries):")
    ])

    # LLM model
    model = Cohere()

    # query generator chain
    query_generator_chain = (
        prompt | model | StrOutputParser() | (lambda x: x.split("\n"))
    )

    # gererate queries
    queries = query_generator_chain.invoke({"original_query": query})

    # add original query
    queries.insert(0, "0. " + query)

    print(queries)

    return queries

In [47]:
def rrf_retriever(query: str) -> list[Document]:
    """RRF retriever

    Args:
        query (str): Query string

    Returns:
        list[Document]: retrieved documents
    """

    # Retriever
    embedding = CohereEmbeddings(model = "embed-english-v3.0")


    qdrant = Qdrant(
        client=qdrant_client,
        collection_name="owners_manual",
        embeddings=embedding,
    )

    retriever = qdrant.as_retriever()

    # RRF chain
    chain = (
        {"query": itemgetter("query")}
        | RunnableLambda(query_generator)
        | retriever.map()
        | reciprocal_rank_fusion
    )

    # invoke
    result = chain.invoke({"query": query})

    return result

In [50]:
# retriever = RunnableLambda(rrf_retriever)
context = rrf_retriever("Explain Hyundai Warranty Policy")

['0. Explain Hyundai Warranty Policy', ' 1. What is the basic warranty coverage on a Hyundai?', '2. What is the length of a Hyundai warranty? ', '3. Are extended warranties available for Hyundai vehicles, and how do they work? ']
results in rrf function:  [[Document(page_content="The table's data describes the Hyundai Warranty Policy and the Road Side Assistance (RSA)\nprogramme that comes with it. The warranty lasts for three years from the date of sale and offers\n24/7 emergency support for Hyundai vehicle owners. In the event of mechanical breakdowns, road\naccidents, or immobility, customers can access roadside assistance by calling a toll-free number.\nThe assistance includes services like roadside repair, vehicle recovery, tire puncture support, and\njump-starting a dead battery. Additionally, it covers key-related issues like locked, lost, or broken\nvehicle keys.\nThe warranty policy explicitly states that the cost of parts replacement and repairs is generally not\ncovered unle

In [51]:
context

[Document(page_content="The table's data describes the Hyundai Warranty Policy and the Road Side Assistance (RSA)\nprogramme that comes with it. The warranty lasts for three years from the date of sale and offers\n24/7 emergency support for Hyundai vehicle owners. In the event of mechanical breakdowns, road\naccidents, or immobility, customers can access roadside assistance by calling a toll-free number.\nThe assistance includes services like roadside repair, vehicle recovery, tire puncture support, and\njump-starting a dead battery. Additionally, it covers key-related issues like locked, lost, or broken\nvehicle keys.\nThe warranty policy explicitly states that the cost of parts replacement and repairs is generally not\ncovered unless specified in the Hyundai Warranty. Customers are advised to visit a nearby\nauthorized Hyundai dealer workshop for assistance. The RSA program's full terms and conditions\ncan be found online at the provided URL.\nOverall, the Hyundai Warranty Policy off

In [53]:
import numpy as np

In [54]:
def calculate_similarity(a, b):
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [55]:
context_list = list()

for i in context:
    context_list.append(i.page_content)

In [56]:
context_list

["The table's data describes the Hyundai Warranty Policy and the Road Side Assistance (RSA)\nprogramme that comes with it. The warranty lasts for three years from the date of sale and offers\n24/7 emergency support for Hyundai vehicle owners. In the event of mechanical breakdowns, road\naccidents, or immobility, customers can access roadside assistance by calling a toll-free number.\nThe assistance includes services like roadside repair, vehicle recovery, tire puncture support, and\njump-starting a dead battery. Additionally, it covers key-related issues like locked, lost, or broken\nvehicle keys.\nThe warranty policy explicitly states that the cost of parts replacement and repairs is generally not\ncovered unless specified in the Hyundai Warranty. Customers are advised to visit a nearby\nauthorized Hyundai dealer workshop for assistance. The RSA program's full terms and conditions\ncan be found online at the provided URL.\nOverall, the Hyundai Warranty Policy offers comprehensive supp

In [59]:
import cohere
import numpy as np

co = cohere.Client(api_key="xxe3X6u8vcTFJgJ8Pc7CfLezwpQiATQcUB56VIUp")

model="embed-english-v3.0"
input_type="search_query"

res = co.embed(texts=context_list,
                model=model,
                input_type=input_type)

In [60]:
res

cohere.Embeddings {
	embeddings: [[0.022766113, -0.011703491, -0.015258789, -0.07354736, -0.01890564, 0.010856628, -0.057678223, 0.014945984, -0.047973633, 0.0077590942, -0.01965332, 0.014678955, -0.035949707, -0.02078247, 0.05215454, -0.04159546, 0.023513794, -0.00504303, -0.0035572052, -0.037628174, 0.009628296, -0.03189087, -0.01008606, 0.01776123, 0.024536133, 0.026733398, -0.019210815, 0.004863739, 0.045043945, 0.024017334, 0.02645874, -0.02557373, 0.0035324097, -0.010513306, 0.0152282715, 0.00057697296, 0.013755798, 0.0118637085, -0.00047016144, 0.019638062, 0.0047950745, -0.006679535, 0.026184082, -0.0066375732, -0.015136719, 0.012687683, 0.003364563, -0.020492554, 0.008110046, 0.024215698, -0.034210205, 0.023803711, -0.03274536, 0.028045654, -0.0043296814, -0.0038719177, -0.06222534, -0.01876831, -0.0058021545, 0.03289795, 0.039611816, 0.00831604, 0.005508423, 0.008178711, -0.0018196106, -0.012260437, 0.045196533, 0.013389587, 0.020614624, -0.006072998, -0.0044288635, 0.0109710

In [63]:
len(res.embeddings)

5

In [64]:
len(res.embeddings[0])

1024

In [66]:
query_emb = co.embed(texts=["Explain Hyundai Warranty Policy"],
                model=model,
                input_type=input_type)

In [67]:
len(query_emb.embeddings)

1

In [69]:
cos_sim(query_emb.embeddings, res.embeddings[0])[0][0]

tensor(0.6396)

In [76]:
def get_response(query, threshold=0.5):
    cohere_client = cohere.Client(api_key="xxe3X6u8vcTFJgJ8Pc7CfLezwpQiATQcUB56VIUp")
    chat_history = []

    context = rrf_retriever("Explain Hyundai Warranty Policy")
    context_list = list()

    for i in context:
        context_list.append(i.page_content)

    co = cohere.Client(api_key="xxe3X6u8vcTFJgJ8Pc7CfLezwpQiATQcUB56VIUp")

    model="embed-english-v3.0"
    input_type="search_query"

    res = co.embed(texts=context_list,
                    model=model,
                    input_type=input_type)

    query_emb = co.embed(texts=[f"{query}"],
                model=model,
                input_type=input_type)
    
    counter = 0
    for i in res:
        if float(cos_sim(query_emb.embeddings, i)[0][0]) < threshold:
            print(float(cos_sim(query_emb.embeddings, i)[0][0]))
            counter += 1

    print(counter)
    prompt = None

    if counter >= 3:
        prompt = f"""
                Create several question based on question:{query}, context: {context} and chat history of the user: {chat_history}.
                As similarity between query and context is low, try to ask several probing questions.
                Ask several followup questions to get further clarity.
                Answer in a polite tone, and convey to the user that you need more clarity to answer the question.
                Then display the probing questions as bulletin points.
                Do not use technical words, give easy to understand responses.
            """
    else:
        prompt = f"""
                Answer the question:{query} only based on the context: {context} and the chat history of the user: {chat_history} provided.
                Try to answer in bulletin points.
                Do not use technical words, give easy to understand responses.
                Do not divulge any other details other than query or context.
            """

    co = cohere.Client("xxe3X6u8vcTFJgJ8Pc7CfLezwpQiATQcUB56VIUp")
    response = co.chat(
        message=prompt,
        model="command-r",
        temperature=0
    )

    return response.text

In [77]:
get_response("Warranty conditions")

['0. Explain Hyundai Warranty Policy', ' 1. What is the basic warranty coverage on a Hyundai?', '2. What are the different types of warranties offered by Hyundai?', "3. Is there a mileage limit on Hyundai's warranty coverage? ", '4. Is there a warranty on a used Hyundai? ', '5. What is the warranty claim process at Hyundai? ', '', "I hope this helps! Let me know if you'd like me to generate any more queries or provide more details on any of these questions. "]
results in rrf function:  [[Document(page_content="The table's data describes the Hyundai Warranty Policy and the Road Side Assistance (RSA)\nprogramme that comes with it. The warranty lasts for three years from the date of sale and offers\n24/7 emergency support for Hyundai vehicle owners. In the event of mechanical breakdowns, road\naccidents, or immobility, customers can access roadside assistance by calling a toll-free number.\nThe assistance includes services like roadside repair, vehicle recovery, tire puncture support, and

"I'd love to help clarify the warranty conditions, but I need more information to understand your specific query. \n\nCould you tell me about the issues you're having with your vehicle or the aspects of the warranty you'd like to clarify? \n\nHere are some initial questions to help us get started:\n\n- Have you had any issues with your Hyundai vehicle that require warranty service? \n- Are you unsure about what repairs or replacements are covered by the Hyundai warranty?\n- Have you purchased any Hyundai replacement parts and need clarification on their warranty? \n\nThese questions might help give me more context, and I can provide more accurate answers accordingly."